Нашей компании нужно сгруппировать клиентов для АБ-тестов. Алгоритм группировки очень простой - взять ID клиента (состоит из 5-7 цифр, например 7412567) и найти сумму всех его цифр. Получившееся число и является номером группы, в которую входит данный клиент.

Для того, чтобы понять, насколько хорош такой простой алгоритм, тебе нужно написать следующие диагностические функции:

1.Функция, которая подсчитывает число покупателей, попадающих в каждую группу, если нумерация ID сквозная и начинается с 0. На вход функция получает целое число n_customers (количество клиентов).

2.Функция, аналогичная первой, если ID начинается с произвольного числа. На вход функция получает целые числа: n_customers (количество клиентов) и n_first_id (первый ID в последовательности).

Напишите код функций на Python и оформите его в репозиторий Github. В форму ответа вставьте ссылку на репозиторий, убедитесь, что он публичный. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame(data = {'user_id':["123456", # создадим примерочную таблицу для проверки
                                     '123456', # создаем дупликаты для проверки
                                     "123465",
                                     "01234343",
                                     "01234343",
                                     "1234874",
                                     "7412567",
                                     "095647",
                                     "095674",
                                     "756123"]})

In [3]:
df

,user_id
0,123456
1,123456
2,123465
3,01234343
4,01234343
5,1234874
6,7412567
7,095647
8,095674
9,756123


In [4]:
df_int = df['user_id'].astype(int).values # переводим колонку с клиентами в 1d array для оптимизации итераций
sum_num = [] # создаем список, куда будем складывать суммы
for value in df_int: # разбив массива на каждый элемент
    sum = 0 # в начале списка сумма сбрасывается, чтобы на каждом начале цикла получать новую сумму
    for digit in str(value): # разбив элемента на отдельный компонент элемента
        sum+=int(digit) # складываем каждый компонент элемента
    sum_num.append(sum) # добавляем рез-тат в список
    

    

In [5]:
sum_num

[21, 21, 21, 20, 20, 29, 32, 31, 31, 24]

In [6]:
df['sum_id'] = pd.Series(sum_num) # добавляем наш список в колонку оригинального дф переводя его в pandas series сначала

In [7]:
df

,user_id,sum_id
0,123456,21
1,123456,21
2,123465,21
3,01234343,20
4,01234343,20
5,1234874,29
6,7412567,32
7,095647,31
8,095674,31
9,756123,24


In [8]:
def id_starts_0(n_customers): # создаем первую функцию проверки (число начинается с 0)
    if n_customers != 0:
        df_1 = df\
        .query('user_id.str.startswith("0")')\
        .drop_duplicates(subset = ['user_id'])\
        .iloc[:n_customers]\
        .groupby('sum_id')\
        .agg({'user_id':"count"}) 

        return df_1
    if n_customers <= 0:
        print(f'{n_customers} is too low for n_customers, try a non-zero, non-negative value.')
        df_1 = pd.DataFrame()
        return df_1
    
# создем новый дф, где сначала фильтруем колонки оригинального дф по условию, что user_id начинается с 0,
# далее отбираем колонки по указанному кол-ву клиентов, удаляем дупликаты user_id, группируем по сумме чисел, а аггрегация по кол-ву
# user_id в каждой группе суммы. Если указывается 0 клиентов или отрицательное число, то функция возвращает текст, где описывается,
# какие значения можно указывать

In [9]:
id_starts_0(n_customers=3)

,user_id
sum_id,
20,1
31,2


In [10]:
id_starts_0(n_customers=0)

0 is too low for n_customers, try a non-zero, non-negative value.


""


In [11]:
def id_starts_not_0(n_customers, n_first_id): # создаем первую функцию проверки (число начинается не с 0)
    if n_customers != 0 and n_first_id != '0':
        df_2 = df\
        .query('user_id.str.startswith(@n_first_id)')\
        .drop_duplicates(subset = ['user_id'])\
        .iloc[:n_customers]\
        .groupby('sum_id')\
        .agg({'user_id':"count"})
        
        return df_2
    
    if n_customers <= 0 and n_first_id == '0':
        print(f'{n_customers} is too low for n_customers, try a non-zero, non-negative value, \nn_first_id of 0 is not intended for this function, use function for user_id starting with 0 value separately')
        
    elif n_customers <= 0:
        print(f'{n_customers} is too low for n_customers, try a non-zero, non-negative value.')
        df_2 = pd.DataFrame()
        return df_2
    elif n_first_id == '0':
        print('n_first_id of 0 is not intended for this function, use function for user_id starting with 0 value separately.')
        df_2 = pd.DataFrame()
        return df_2
   

# создаем новый дф, где сначала фильтруем колонки оригинального дф по условию, что user_id начинается с указанного числа (но не 0),
# далее отбираем колонки по указанному кол-ву клиентов, удаляем дупликаты user_id, группируем по сумме чисел, а аггрегация по кол-ву
# user_id в каждой группе суммы.Если указывается 0 клиентов или отрицательное число, то функция возвращает текст, где описывается,
# какие значения можно указывать. Если n_first_id = 0, то тоже выдаем инструкции пользоваться другой функцией, предназначенной для 
# значений, начинающихся с 0.

In [12]:
id_starts_not_0(n_customers = 2, n_first_id = '1')

,user_id
sum_id,
21,2


In [13]:
id_starts_not_0(n_customers = 0, n_first_id = '1')

0 is too low for n_customers, try a non-zero, non-negative value.


""


In [14]:
id_starts_not_0(n_customers = 0, n_first_id = '0')

0 is too low for n_customers, try a non-zero, non-negative value, 
n_first_id of 0 is not intended for this function, use function for user_id starting with 0 value separately


In [15]:
id_starts_not_0(n_customers = 1, n_first_id = '0')

n_first_id of 0 is not intended for this function, use function for user_id starting with 0 value separately.


""
